# Roandom Forest

## 0. Setup

### 0.1. Install packages

In [ ]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
#!pip install panelsplit
#!pip install openpyxl

### 0.2. Load packages

In [1]:
# from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
import random

from panelsplit.cross_validation import PanelSplit

import shap
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import os
import openpyxl

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import TimeSeriesSplit  # Alternative to PanelSplit

c:\Users\mmier\anaconda3\envs\MDS_thesis\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 0.3. Load Data

In [3]:
### Check working directory
#print(os.getcwd()) 

### Define file and path
file_path = r"c:\Users\mmier\OneDrive - Hertie School\3. Estudio\2025 MDS\2025-1 MDS Thesis\MDS_thesis\Data\OSC" #Use a raw string (r"") when defining paths
file = "Datos-ICM-2023.xlsx"
full_path = os.path.join(file_path, file)

### List files in directory
#print(os.listdir(file_path))

### Load excel file
df = pd.read_excel(full_path, engine="openpyxl")

# Display the first few rows
#print(df.head())

### 0.4. Correct df format

In [4]:
### Rename columns names with row 4
df.columns = df.iloc[3]

### Delete first (index 0) and third (index 2) row
df = df.drop([0, 1, 2, 3], axis=0)

### Reset index 
df = df.reset_index(drop=True)


## 1. Preprocessing

### 1.1. Define train, test and evaluation set

Evaluation set: 2019 to 2022

In [5]:
### Separate evaluation set 
final_df = df[df["AÑO"] < 2019]

### 1.2. Define y and X1

y: deforestation

M-03-25	Hectáreas de bosque deforestadas

X1: general variables

Aglomeración
SC Función ciudades
Departamento
Municipio
Divipola
AÑO
ICM-00-0	Índice de Ciudades Modernas
PCC-00-0	Índice de Productividad, Competitividad y Complementariedad Económica
GPI-00-0	Índice de Gobernanza, Participación e Instituciones
EIS-00-0	Índice de Equidad e inclusión social
CTI-00-0	Índice de Ciencia, Tecnología e Innovación
SEG-00-0	Índice de Seguridad
SOS-00-0	Índice de Sostenibilidad

In [21]:
### Define y
y = final_df["M-03-25"]

### Define X1: general variables 
X1 = final_df[["Aglomeración", "SC Función ciudades", "Departamento", "Municipio", "Divipola", "AÑO","ICM-00-0", "PCC-00-0", "GPI-00-0", "EIS-00-0", "CTI-00-0", "SEG-00-0", "SOS-00-0"]]

### 1.3. Remove NAS and order by "Municipio" and "AÑO"

Prevents future values from being included in training accidentally.

In [22]:
### Check for missing values
#print(f"Missing values in X1:\n{X1.isnull().sum().sum()}")
#print(f"Missing values in y:\n{y.isnull().sum().sum()}")

### Remove NAS
df_combined = pd.concat([X1, y], axis=1)  # Combine into one DataFrame
df_combined.dropna(inplace=True)  # Remove rows with any NaN values
df_combined = df_combined.reset_index(drop=True) # Reset index 

### Organize by municipality and year for time series split 
df_combined = df_combined.sort_values(by=["Municipio", "AÑO"]).reset_index(drop=True)

### Separate again
X1 = df_combined.iloc[:, :-1]  # All columns except the last (features)
y = df_combined.iloc[:, -1]    # The last column (target variable)

### 1.4. Encode for  

In [23]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

for col in ["Aglomeración", "SC Función ciudades", "Departamento", "Municipio", "Divipola"]:
    le = LabelEncoder()
    X1[col] = le.fit_transform(X1[col])
    
print(y.dtype)
print(X1["Aglomeración"].dtype)
print(X1["SC Función ciudades"].dtype)
print(X1["Departamento"].dtype)
print(X1["Municipio"].dtype)
print(X1["Divipola"].dtype)



int64
int64
int64
int64
int64
int64


C:\Users\mmier\AppData\Local\Temp\ipykernel_21044\3457745230.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X1[col] = le.fit_transform(X1[col])
C:\Users\mmier\AppData\Local\Temp\ipykernel_21044\3457745230.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X1[col] = le.fit_transform(X1[col])
C:\Users\mmier\AppData\Local\Temp\ipykernel_21044\3457745230.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

### 1.4. Create lags

## 2. Random Forest Model

In [24]:
### Set seed for reproducibility
seed_value = 17
np.random.seed(seed_value)

### Define Time-Series Cross-Validation (5 splits)

tscv = TimeSeriesSplit(n_splits=5)

### Hyperparameters for Random Forest
n_estimators_values = [50, 100, 200, 300]  # Number of trees to test
max_depth_values = [5, 10, 15, 20, 30, None]  # Depth of trees

### Function to compute Adjusted R²
def adjusted_r2(r2, n, k):
    return 1 - (1 - r2) * ((n - 1) / (n - k - 1))

### Dictionary to store results
results = {}



### Perform Time-Series Cross-Validation (iterate over each split)
for train_idx, test_idx in tscv.split(X1):

    ## Split dataset into train & test per fold
    X1_train, X1_test = X1.iloc[train_idx], X1.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    ## Train and evaluate for each combination of hyperparameters
    for n_estimators in n_estimators_values:
        for max_depth in max_depth_values:
            
            # Define the Random Forest model
            model = RandomForestRegressor(
                n_estimators=n_estimators,
                max_depth=max_depth,
                random_state=seed_value,
                n_jobs=-1  # Use all available processors
            )
            # Train the model
            model.fit(X1_train, y_train.ravel())

            # Predictions
            y_train_pred = model.predict(X1_train)
            y_test_pred = model.predict(X1_test)

            # Compute performance metrics
            mse = mean_squared_error(y_test, y_test_pred)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test, y_test_pred)
            r2_test = r2_score(y_test, y_test_pred)
            r2_train = r2_score(y_train, y_train_pred)

            # Compute Adjusted R²
            n_train, k = X1_train.shape
            n_test = X1_test.shape[0]
            adj_r2_train = adjusted_r2(r2_train, n_train, k)
            adj_r2_test = adjusted_r2(r2_test, n_test, k)

            # Store results for this combination
            results[(n_estimators, max_depth)] = {
                "MSE": mse, "RMSE": rmse, "MAE": mae, "R2_test": r2_test, "R2_train": r2_train,
                "Adj_R2_test": adj_r2_test, "Adj_R2_train": adj_r2_train
            }
            
### Find the best hyperparameter combination (minimize MSE, maximize R²)
best_params = min(results, key=lambda x: (results[x]["MSE"], -results[x]["R2_test"]))
best_metrics = results[best_params]

### Print optimal hyperparameters and their performance
print(f"🌲 Optimal Random Forest Parameters: n_estimators={best_params[0]}, max_depth={best_params[1]}")
print(f"📊 Best MSE: {best_metrics['MSE']:.4f}")
print(f"📊 Best RMSE: {best_metrics['RMSE']:.4f}")
print(f"📊 Best MAE: {best_metrics['MAE']:.4f}")
print(f"📊 R² (Train): {best_metrics['R2_train']:.4f}, Adjusted R² (Train): {best_metrics['Adj_R2_train']:.4f}")
print(f"📊 R² (Test): {best_metrics['R2_test']:.4f}, Adjusted R² (Test): {best_metrics['Adj_R2_test']:.4f}")



AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

## Correction suggested

In [ ]:
### Set seed for reproducibility
seed_value = 17
np.random.seed(seed_value)

### Define Time-Series Cross-Validation (5 splits)
tscv = TimeSeriesSplit(n_splits=5, max_train_size=None)

### Hyperparameters for Random Forest
n_estimators_values = [50, 100, 200, 300]  # Number of trees to test
max_depth_values = [5, 10, 20, None]  # Depth of trees


### Function to compute Adjusted R²
def adjusted_r2(r2, n, k):
    if n <= k + 1:
        return np.nan  # Avoid division by zero
    return 1 - (1 - r2) * ((n - 1) / (n - k - 1))

### Dictionary to store results
results = {}

### Perform Time-Series Cross-Validation
for train_idx, test_idx in tscv.split(X1):
    print(f"Train size: {len(train_idx)}, Test size: {len(test_idx)}")  # Debug check

    ## Split dataset into train & test per fold (using `.loc[]` for safe indexing)
    X1_train, X1_test = X1.loc[X1.index[train_idx]], X1.loc[X1.index[test_idx]]
    y_train, y_test = y.loc[y.index[train_idx]], y.loc[y.index[test_idx]]

    ## Train and evaluate for each combination of hyperparameters
    for n_estimators in n_estimators_values:
        for max_depth in max_depth_values:
            
            # Define the Random Forest model
            model = RandomForestRegressor(
                n_estimators=n_estimators,
                max_depth=max_depth,
                random_state=seed_value,
                n_jobs=-1  # Use all available processors
            )
            
            # Train the model
            model.fit(X1_train, y_train)

            # Predictions
            y_train_pred = model.predict(X1_train)
            y_test_pred = model.predict(X1_test)

            # Compute performance metrics
            mse = mean_squared_error(y_test, y_test_pred)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test, y_test_pred)
            r2_test = r2_score(y_test, y_test_pred)
            r2_train = r2_score(y_train, y_train_pred)

            # Compute Adjusted R²
            n_train, k = X1_train.shape
            n_test = X1_test.shape[0]
            adj_r2_train = adjusted_r2(r2_train, n_train, k)
            adj_r2_test = adjusted_r2(r2_test, n_test, k)

            # Store results for this combination (append results for all folds)
            if (n_estimators, max_depth) not in results:
                results[(n_estimators, max_depth)] = []

            results[(n_estimators, max_depth)].append({
                "MSE": mse, "RMSE": rmse, "MAE": mae, "R2_test": r2_test, "R2_train": r2_train,
                "Adj_R2_test": adj_r2_test, "Adj_R2_train": adj_r2_train
            })

### Compute Average Performance Across Folds
averaged_results = {
    params: {metric: np.mean([fold[metric] for fold in folds])
             for metric in folds[0].keys()}
    for params, folds in results.items()
}

### Find the best hyperparameter combination (minimize MSE, maximize R²)
best_params = min(averaged_results, key=lambda x: (averaged_results[x]["MSE"], -averaged_results[x]["R2_test"]))
best_metrics = averaged_results[best_params]

### Print optimal hyperparameters and their performance
print(f"🌲 Optimal Random Forest Parameters: n_estimators={best_params[0]}, max_depth={best_params[1]}")
print(f"📊 Best MSE: {best_metrics['MSE']:.4f}")
print(f"📊 Best RMSE: {best_metrics['RMSE']:.4f}")
print(f"📊 Best MAE: {best_metrics['MAE']:.4f}")
print(f"📊 R² (Train): {best_metrics['R2_train']:.4f}, Adjusted R² (Train): {best_metrics['Adj_R2_train']:.4f}")
print(f"📊 R² (Test): {best_metrics['R2_test']:.4f}, Adjusted R² (Test): {best_metrics['Adj_R2_test']:.4f}")


Train size: 1686, Test size: 1681
Train size: 3367, Test size: 1681
Train size: 5048, Test size: 1681
Train size: 6729, Test size: 1681
Train size: 8410, Test size: 1681
🌲 Optimal Random Forest Parameters: n_estimators=50, max_depth=5
📊 Best MSE: 891004.4595
📊 Best RMSE: 908.7719
📊 Best MAE: 191.9636
📊 R² (Train): 0.4441, Adjusted R² (Train): 0.4432
📊 R² (Test): -0.0143, Adjusted R² (Test): -0.0186


## 5. Again

### 5.1. Implement GroupKFold for Panel Data

We need to group by "Municipio" when performing cross-validation.

- Ensures that all data from one municipality is only in train or test (not both).
- Respects time structure by sorting by "AÑO" first.
- Prevents municipalities from leaking into both train and test sets.

In [27]:
# Ensure y is a Pandas Series (not a NumPy array)
if isinstance(y, np.ndarray):  
    y = pd.Series(y, index=X1.index)  # Restore indexing



from sklearn.model_selection import GroupKFold

# Define the number of splits
n_splits = 5  
tscv = GroupKFold(n_splits=n_splits)

# Extract groups (Municipio column as identifier)
groups = X1["Municipio"].values  

for train_idx, test_idx in tscv.split(X1, y, groups=groups):
    X1_train, X1_test = X1.iloc[train_idx], X1.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    print(f"Train size: {len(train_idx)}, Test size: {len(test_idx)}")


Train size: 8075, Test size: 2016
Train size: 8075, Test size: 2016
Train size: 8069, Test size: 2022
Train size: 8076, Test size: 2015
Train size: 8069, Test size: 2022


### 5.2. Train a Quick Random Forest to Identify Important Features

Before running the full model, we train a baseline Random Forest to rank feature importance.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

# Define a baseline Random Forest model
rf_baseline = RandomForestRegressor(
    n_estimators=50,  # Small number of trees for quick evaluation
    max_depth=10,     # Limit tree depth to avoid overfitting
    random_state=17,
    n_jobs=-1         # Use all CPU cores
)

# Fit the model on the full dataset (ignoring "AÑO" and "Municipio" for now)
X1_selected = X1.drop(columns=["Aglomeración", "SC Función ciudades", "Departamento", "Municipio", "Divipola", "AÑO"])  # Remove non-predictive ID variables
rf_baseline.fit(X1_selected, y)

# Extract feature importance
feature_importance_df = pd.DataFrame({
    "Feature": X1_selected.columns,
    "Importance": rf_baseline.feature_importances_
}).sort_values(by="Importance", ascending=False)

# Display the top 30 features
print(feature_importance_df.head(30))
#import ace_tools as tools
#tools.display_dataframe_to_user(name="Top Feature Importance", dataframe=feature_importance_df.head(3))


    Feature  Importance
0  ICM-00-0    0.395368
6  SOS-00-0    0.166915
1  PCC-00-0    0.109436
2  GPI-00-0    0.103235
3  EIS-00-0    0.090039
5  SEG-00-0    0.069330
4  CTI-00-0    0.065677


### 5.3. Features 

In [ ]:
# Keep only the top 30 most important features
#top_features = feature_importance_df["Feature"].head(30).tolist()

# Update X1 to only include the top features
#X1 = X1[top_features]

#print(f"✅ Selected top {len(top_features)} features for the final model.")


array([0.39536799, 0.10943638, 0.10323458, 0.09003945, 0.06567688,
       0.06932975, 0.16691497])

### 5.4. Train the Final Random Forest Model

Train Random Forest with Cross-Validation

In [38]:
# Define hyperparameters for tuning
n_estimators_values = [50, 100, 200, 300]  # Number of trees
max_depth_values = [5, 10, 20, None]       # Depth of trees

# Store results
results = {}

# Perform Time-Series Cross-Validation
from sklearn.model_selection import GroupKFold
tscv = GroupKFold(n_splits=5)
groups = X1.index  # Using index since Municipio was dropped

for train_idx, test_idx in tscv.split(X1, y, groups=groups):
    X1_train, X1_test = X1.iloc[train_idx], X1.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Train and evaluate for each combination of hyperparameters
    for n_estimators in n_estimators_values:
        for max_depth in max_depth_values:
            
            # Define the Random Forest model
            model = RandomForestRegressor(
                n_estimators=n_estimators,
                max_depth=max_depth,
                random_state=17,
                n_jobs=-1  # Use all available processors
            )
            
            # Train the model
            model.fit(X1_train, y_train)

            # Predictions
            y_train_pred = model.predict(X1_train)
            y_test_pred = model.predict(X1_test)

            # Compute performance metrics
            from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
            mse = mean_squared_error(y_test, y_test_pred)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test, y_test_pred)
            r2_test = r2_score(y_test, y_test_pred)
            r2_train = r2_score(y_train, y_train_pred)

            # Store results
            if (n_estimators, max_depth) not in results:
                results[(n_estimators, max_depth)] = []

            results[(n_estimators, max_depth)].append({
                "MSE": mse, "RMSE": rmse, "MAE": mae, "R2_test": r2_test, "R2_train": r2_train
            })

### Compute Average Performance Across Folds
averaged_results = {
    params: {metric: np.mean([fold[metric] for fold in folds])
             for metric in folds[0].keys()}
    for params, folds in results.items()
}

### Find the best hyperparameter combination
best_params = min(averaged_results, key=lambda x: (averaged_results[x]["MSE"], -averaged_results[x]["R2_test"]))
best_metrics = averaged_results[best_params]

### Print optimal hyperparameters and performance
print(f"🌲 Best Random Forest Parameters: n_estimators={best_params[0]}, max_depth={best_params[1]}")
print(f"📊 Best MSE: {best_metrics['MSE']:.4f}")
print(f"📊 Best RMSE: {best_metrics['RMSE']:.4f}")
print(f"📊 Best MAE: {best_metrics['MAE']:.4f}")
print(f"📊 R² (Train): {best_metrics['R2_train']:.4f}")
print(f"📊 R² (Test): {best_metrics['R2_test']:.4f}")


🌲 Best Random Forest Parameters: n_estimators=300, max_depth=None
📊 Best MSE: 1285427.8743
📊 Best RMSE: 1133.7549
📊 Best MAE: 877.6061
📊 R² (Train): 0.9542
📊 R² (Test): 0.6917
